# **Project 2: The Dijkstra's Algorithm**

| **Group Member** | **Matriculation Number** |
|---------|-------|
| Jyoshika Barathimogan | |
| Kong Fook Wah | U2421655E |
| Kris Khor Hai Xiang | U2421377C |

## **Table of Content**

- [Background]
- [Purpose]
- [Implementation]
- [Result]
- [Conclusion]


## Purpose of Project 2

This project aims to determine how the choice representations for input graph and priority queue will affect the time complexity of Dijkstra's Algorithm.

## **1. Reproducibility & Constant Variable**

In [ ]:
import heapq
import random
import networkx as nx
import matplotlib.pyplot as plt
import time
import numpy as np
import math

INF = float('inf')

## Helper Class to conduct the experiment

In [ ]:
class Helper:
    def visualiseGraph(graph):
        """
        Visualize a graph from either adjacency list or adjacency matrix format.
        
        Args:
            graph: Either adjacency list (list of lists of tuples) 
                or adjacency matrix (2D list)
        """
        G = nx.Graph()
        
        # Detect format: check if first element is a list (matrix) or contains tuples (list)
        if graph and isinstance(graph[0], list):
            # Check if it's a matrix (contains numbers/INF) or adjacency list (contains tuples)
            if graph[0] and isinstance(graph[0][0], (int, float)):
                # Adjacency Matrix format
                INF = float('inf')
                for u in range(len(graph)):
                    for v in range(u + 1, len(graph[u])):
                        if graph[u][v] != INF and graph[u][v] != 0:
                            G.add_edge(u, v, weight=graph[u][v])
            else:
                # Adjacency List format
                for u in range(len(graph)):
                    for v, w in graph[u]:
                        if u < v:
                            G.add_edge(u, v, weight=w)
        
        pos = nx.spring_layout(G, seed=42)
        plt.figure(figsize=(6, 6))
        nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=1500,
                edge_color='gray', font_size=10, font_weight='bold')
        edge_labels = nx.get_edge_attributes(G, 'weight')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
        plt.show()

    def printAdjacencyMatrix(graph):
        """
        Print adjacency matrix with row and column headers.
        
        Args:
            graph: 2D adjacency matrix
        """
        size = len(graph)
        INF = float('inf')
        
        # Determine column width based on largest value
        max_width = max(len(str(graph[i][j]) if graph[i][j] != INF else '∞') 
                        for i in range(size) for j in range(size))
        max_width = max(max_width, 3)  # minimum width of 3
        
        # Print column headers
        print("    ", end="")
        for j in range(size):
            print(f"{j:>{max_width}}", end=" ")
        print()
        
        # Print separator line
        print("   " + "-" * (size * (max_width + 1) + 1))
        
        # Print rows with row headers
        for i in range(size):
            print(f"{i:2} |", end=" ")
            for j in range(size):
                if graph[i][j] == INF:
                    print(f"{'∞':>{max_width}}", end=" ")
                else:
                    print(f"{graph[i][j]:>{max_width}}", end=" ")
            print()

    def printAdjacencyList(graph):
        """
        Print adjacency list in a clean, readable format.
        
        Args:
            graph: List of lists containing (vertex, weight) tuples
        """
        print("Vertex | Adjacent Vertices (weight)")
        print("-------+---------------------------")
        
        for i in range(len(graph)):
            print(f"  {i:2}   | ", end="")
            if graph[i]:
                edges = [f"({v}, {w})" for v, w in sorted(graph[i])]
                print(", ".join(edges))
            else:
                print("(no edges)")
        print()


## **2. Implementation of Dijkstra's Algorithm**

# PART 1 (Adjacency Matrix + Array Based Priority Queue)

In [ ]:
class Counter1:
    def __init__(self):
        self.operations = 0

In [ ]:
class PriorityQueueArray:
    def __init__(self, counter=None):
        self.queue = []   # will store (distance, vertex)
        self.counter = counter

    def __len__(self):
        return len(self.queue)

    def push(self, item):
        self.queue.append(item)

    def pop(self):
        min_index = 0
        for i in range(1, len(self.queue)):
            self.counter.operations += 1
            if self.queue[i][0] < self.queue[min_index][0]:
                min_index = i
        return self.queue.pop(min_index)


In [ ]:
def dijkstraMatrix_ArrayPQ(graph, source=0):
    n = len(graph)
    counter = Counter1()
    pq = PriorityQueueArray(counter)   # our array-based PQ
    dist = [INF] * n
    parent = [None] * n

    dist[source] = 0
    pq.push((0, source))   # (distance, vertex)

    while len(pq) > 0:
        curr_dist, u = pq.pop()
        if curr_dist > dist[u]:
            continue

        for v in range(n):   # check all vertices
            weight = graph[u][v]
            counter.operations += 1

            if weight != INF and weight != 0:   # there is an edge
                if dist[u] + weight < dist[v]:
                    dist[v] = dist[u] + weight
                    parent[v] = u
                    pq.push((dist[v], v))

    return dist, parent, counter


# Use the Adjacency Matrix traversal (for v in range(n) instead of for v, weight in graph[u]).

## Generate adjacency Matrix Graph

In [ ]:
def generateAdjacencyMatrixGraph(size, density=0.5, wmin=1, wmax=20):
    INF = float('inf')
    graph = [[INF] * size for _ in range(size)]

    for i in range(size):
        graph[i][i] = 0
        for j in range(i+1, size):
            if random.random() < density:  # randomly decide if edge exists
                w = random.randint(wmin, wmax)
                graph[i][j] = w
                graph[j][i] = w

    # ensure every vertex has at least one edge
    for i in range(size):
        if all(graph[i][j] == INF for j in range(size) if j != i):
            j = random.choice([x for x in range(size) if x != i])
            w = random.randint(wmin, wmax)
            graph[i][j] = w
            graph[j][i] = w

    return graph


In [ ]:
test_range = range(2, 200)
mean_execution_time = []
mean_operations = []
trials = 10

for i in test_range:
    
    total_time = 0
    total_operations = 0

    for _ in range(trials):
        graph = generateAdjacencyMatrixGraph(i)   # matrix generator
        start_time = time.perf_counter()

        _, _ , counter = dijkstraMatrix_ArrayPQ(graph, source=0)   # matrix + array PQ

        end_time = time.perf_counter()
        total_time += (end_time - start_time)
        total_operations += counter.operations

    mean_execution_time.append(total_time / trials)
    mean_operations.append(total_operations/trials)

# Plotting

test_range_array = np.array(list(test_range))
V2 = test_range_array**2
c_ops = (V2 @ mean_operations) / (V2 @ V2)  # solve min ||c*V^2 - ops||
theoretical_v2 = c_ops * V2

plt.figure(figsize=(10, 10))
plt.plot(test_range, mean_operations, linestyle='--', label='Mean Operations Count', color='red')
plt.plot(test_range_array, theoretical_v2, linestyle='-', label='O(V²) theoretical', color='darkred')
plt.xlabel('Number of Nodes')
plt.ylabel('Mean Operations Count')
plt.title('Mean Operations Count vs Number of Nodes')
plt.legend()

plt.figure(figsize=(10, 10))
plt.plot(test_range, mean_execution_time, linestyle='--', label='Execution Time vs Number of Nodes')
plt.xlabel('Number of Nodes')
plt.ylabel('Mean Execution Time (s)')
plt.title('Execution Time vs Number of Nodes (Matrix + Array PQ)')
plt.legend()


plt.show()


# PART 2 ( Adjacency Matrix + Min-Heap Priority Queue)

In [ ]:
class Counter2:
    def __init__(self):
        self.adj_checks = 0        # iterating neighbors
        self.relax_checks = 0      # if dist[u]+w < dist[v]
        self.relax_success = 0     # relaxations that happened
        self.heap_ops = 0          # estimated heap "log" work

    @property
    def operations(self):
        return self.adj_checks + self.relax_checks + self.relax_success + self.heap_ops

In [ ]:
class PriorityQueueMinHeap:
    def __init__(self, counter):
        self.queue = []
        self.counter = counter
        heapq.heapify(self.queue)
        
    def __len__(self):
        return len(self.queue)
    
    def push(self, item):
        n = len(self.queue) + 1

        if n <= 1:                          #if only root node in min heap
            self.counter.heap_ops += 1
        else: 
            self.counter.heap_ops += math.ceil(math.log2(n))

        heapq.heappush(self.queue, item)

    def pop(self):
        n = len(self.queue)
        if n <= 1:                          #if only root node in min heap
            self.counter.heap_ops += 1
        else: 
            self.counter.heap_ops += math.ceil(math.log2(n))

        shortest = heapq.heappop(self.queue)
        return shortest

In [ ]:
def dijkstraLists_MinHeapPQ(graph, source=0):
    n = len(graph)
    counter = Counter2()
    pq = PriorityQueueMinHeap(counter)
    dist = [INF] * n
    parent = [None] * n
    visited = [False] * n

    dist[source] = 0
    pq.push((0, source))

    while len(pq) > 0:
        curr_dist, u = pq.pop()
        if curr_dist > dist[u] or visited[u]:
            continue
        
        visited[u] = True
        
        for v, weight in graph[u]:
            
            counter.adj_checks += 1
            counter.relax_checks += 1

            if dist[u] + weight < dist[v]:
                dist[v] = dist[u] + weight
                parent[v] = u

                counter.relax_success += 1
                pq.push((dist[v], v))

    return dist, parent, counter

## Generate adjacency lists Graph

In [ ]:
def generateAdjacencyListsGraph(size, density=0.5, wmin=1, wmax=20):
    graph = [[] for _ in range(size)]

    for i in range(size):
        for j in range(i+1, size):
            if random.random() < density:   #to ensure complete randomness on whether the edge exists, without this, every vertices will be connected
                w = random.randint(wmin, wmax)
                graph[i].append((j, w))
                graph[j].append((i, w))

    #To ensure every vertex has at least one edge (to prevent unreachable vertex situation)

    for i in range(size):
        if not graph[i]:                                             # if the particular vertex has no edge
            j = random.choice([x for x in range(size) if x != i])    #choose a random vertex to connect with that vertex
            w = random.randint(wmin, wmax)
            graph[i].append((j, w))
            graph[j].append((i, w))

    return graph

In [ ]:
test_range = range(2, 200)
mean_execution_time = []
mean_operations = []
trials = 10

for i in test_range:

    total_time = 0
    total_operations = 0

    for _ in range(trials):
        graph = generateAdjacencyListsGraph(i)
        start_time = time.perf_counter()

        _, _, counter = dijkstraLists_MinHeapPQ(graph)

        end_time = time.perf_counter()

        total_time = end_time - start_time
        total_operations += counter.operations

    mean_operations.append(total_operations/trials)
    mean_execution_time.append(total_time/trials)



# --- Theoretical O((V + E) log V) curve ---
density = 0.5
test_range_array = np.array(list(test_range))
E_est = density * (test_range_array * np.log2(test_range_array))
theoretical_ops = (test_range_array + E_est) * np.log2(test_range_array)

# Fit a constant to scale the theoretical curve roughly to your data
c = (theoretical_ops @ mean_operations) / (theoretical_ops @ theoretical_ops)
theoretical_curve = c * theoretical_ops

plt.figure(figsize=(10,10))

plt.plot(test_range, mean_operations, linestyle='--', label='Mean Operations Count', color='red')
plt.plot(test_range_array, theoretical_curve, linestyle='-', label='O((V+E) log V) theoretical', color='darkred')
plt.xlabel('Number of Nodes')
plt.ylabel('Mean Operations Count')
plt.title('Mean Operations Count vs Number of Nodes')
plt.legend()

plt.figure(figsize=(10, 10))
plt.plot(test_range, mean_execution_time, linestyle='--', label='Execution Time vs Number of Nodes')
plt.xlabel('Number of Nodes')
plt.ylabel('Mean Execution Time')
plt.title('Execution Time vs Number of Nodes ')
plt.legend()

plt.show()

# PART 3

In [ ]:
def compareImplementations(test_range=range(2,200), trials=5):
    mean_matrix = []
    mean_list = []

    for i in test_range:
        total_matrix = 0
        total_list = 0

        for _ in range(trials):
            # Matrix + Array PQ
            g_matrix = generateAdjacencyMatrixGraph(i)
            start = time.perf_counter()
            dijkstraMatrix_ArrayPQ(g_matrix)
            end = time.perf_counter()
            total_matrix += (end - start)

            # List + MinHeap PQ
            g_list = generateAdjacencyListsGraph(i)
            start = time.perf_counter()
            dijkstraLists_MinHeapPQ(g_list)
            end = time.perf_counter()
            total_list += (end - start)

        mean_matrix.append(total_matrix / trials)
        mean_list.append(total_list / trials)

    # Plotting both
    plt.figure(figsize=(10, 10))
    plt.plot(test_range, mean_matrix, linestyle='--', label='Matrix + Array PQ (O(V²))')
    plt.plot(test_range, mean_list, linestyle='-', label='List + Min-Heap PQ (O((V+E) log V))')
    plt.xlabel('Number of Nodes')
    plt.ylabel('Mean Execution Time (s)')
    plt.title('Comparison of Dijkstra Implementations')
    plt.legend()
    plt.show()


In [ ]:
compareImplementations()